# Impact of texture in 2D powder diffraction patterns

## Using filters during 2D data integration

In this notebook, we demonstrate the impact of texture when integrating 2D diffraction patterns.

This texture arises from large crystals present along the X-ray beam path. If these crystals are relatively large compared to the illuminating X-ray beam size, then high intensity diffraction spots will appear in the acquired 2D diffraction patterns. Integrating these "spotty" 2D diffraction patterns can lead to major issues:

1) The relative intensities in the integrated 1D diffraction pattern are arbitrary and cannot be modelled using a CIF. This is often solved by using preferred orientation but this does not describe the real phenomenon and any quantitative results regarding weight fractions in multicomponent systems have no value.

2) In X-ray diffraction computed tomography, these will translate to spots in the sinogram data and eventually to line/streak artefacts in the reconstructed image. A more detailed study regarding the usage of filters to suppress single crystal artefacts in X-ray diffraction computed tomography (XRD-CT) can be found here: https://scripts.iucr.org/cgi-bin/paper?nb5157 

A solution to this problem is to apply filters to the raw 2D diffraction patterns prior to data integration. These filters should be applied in the polar transformed images, not the raw 2D diffraction patterns. They supress the spotiness and convert the signel to more powder diffraction-like.

In this notebook, we will look a simple example to demonstrate this. First, let's import the required modules

In [ ]:
import pyFAI, h5py
import numpy as np
import matplotlib.pyplot as plt

Now provide the path to the data

In [ ]:
p = '../../../examples/xrd2D/'

Now read the poni calibration file and the two images

In [ ]:
poni = "%sSpotty_calib.poni" %p
ai = pyFAI.load(poni)

data = "%sSpotty.h5" %p
with h5py.File(data, 'r') as f:
    im_powder = f['powder'][:]
    im_spotty = f['spotty'][:]
    

Now let's plot the two images

In [ ]:
plt.figure(1, figsize=(20,20));plt.clf()
plt.imshow(im_powder, cmap='gray')
plt.clim(0, 500)
plt.show()

plt.figure(2, figsize=(20,20));plt.clf()
plt.imshow(im_spotty, cmap='gray')
plt.clim(0, 500)
plt.show()

It can clearly be seen that the second 2D diffraction pattern is very spotty. It is possible to suppress this spottiness by applying an trimmed mean filter. Here, we apply a 30 % trimmed mean filter and compare the three integrated diffraction patterns: 1) the spots-free pattern after azimuthal integration, 2) the spotty pattern after azimuthal integration without filter and 3) the spotty pattern after azimuthal integration using a 30% trimmed mean filter.

In [ ]:
npt_rad = 1024

tth, I = ai.integrate1d(data=im_powder, npt=npt_rad, unit="2th_deg")
tth, I_spotty = ai.integrate1d(data =im_spotty, npt=npt_rad, unit="2th_deg")
tth_filt, I_filt = ai.medfilt1d(data=im_spotty, npt_rad=npt_rad, unit="2th_deg", percentile=(15,85))

plt.figure(2, figsize=(15,15));plt.clf()
plt.plot(tth,I)
plt.plot(tth,I_spotty)
plt.plot(tth_filt,I_filt)
plt.legend(['Powder', 'Spotty - No filter', 'Spotty - 30% trimmed mean filter'])
plt.xlim(12, 25)
plt.ylim(0, 100)
plt.show()


We can see that the application of the trimmed mean filter leads to a 1D diffraction pattern that is much more similar to the powder pattern, in terms of relative peak intensities, compared to the one obtained without any filters.

In the collage below, we can see four image reconstructions (bottom row) from four corresponding sinograms (top row). It can be clearly seen that the spots present in the BaWO4 sinogram lead to line or strak artefacts in the reconstructed image.

![title](../_static/filt_1.png)

The application of a light trimmed mean filter (3% in this case) in the raw 2D diffraction patterns during integration (polar transformed images) was able to remove these spots and lead to artefact-free reconstructed images.

![title](../_static/filt_2.png)

A second example is provided in the collage below. It can be seen here that all sinograms suffer at some extent from the presence of spots.

![title](../_static/filt_3.png)

Similarly to the previous example, the application of a light trimmed mean filter was able to remove these spots and lead to artefact-free reconstructed images. It is now possible to observe the spatial distribution of all phases.

![title](../_static/filt_4.png)